# Part II: Model-based Regularization

We now want to solve the problem

$$ \min_{u} \frac{1}{2}\|Au - k\|^2 + \operatorname{TV}(u)$$

where $TV:?\to ?$ is defined as

Becaus hard solve this that

$$
\min_{u}
\Big[\max_{y} \langle Au - k_t, y  \rangle - \frac{1}{2} \|y\|^2 \big] +
\Big[
\max_{z} \langle \nabla u, z \rangle - \chi_{B^\infty_\lambda}(z)
\Big]
$$

Employing forward-backward splitting for $u,y,z$ yields the update

$$
\begin{align*}
y \gets \frac{1}{1+\sigma}y + \frac{\sigma}{1+\sigma}(Au  -k_t )\\
z \gets \operatorname{proj}_{B^\infty_\lambda}\left( z + \sigma\nabla u\right)\\
u^-\gets u\\
u\gets u - \tau (A^Ty - \operatorname{div}(z))
\end{align*}
$$

Additionally we perform an overrelaxation step for $u$, 

$$
u\gets 2 u - u^-.
$$

In [1]:
import skimage as ski
import numpy as np

num_theta = 150
dim = 150
noise_lvl = 0.01 * dim
phantom = ski.img_as_float(ski.data.shepp_logan_phantom())
phantom = ski.transform.resize(phantom, (dim, dim))
theta = np.linspace(0,180, endpoint = False, num=num_theta)
sinogram =  ski.transform.radon(phantom, theta)
sinogram += np.random.normal(0, noise_lvl, size=sinogram.shape)

In [2]:
from optimizer import pdhg, imgrad, imdiv
from optimizer import lifted_variable as lv

class Radon:
    def __call__(self, u):
        return ski.transform.radon(u, theta)*(np.pi/(2 * num_theta))
    
    def adjoint(self, k):
        return ski.transform.iradon(k, theta, filter_name=None)

class Grad:
    def __call__(self, u):
        return imgrad(u)

    def adjoint(self, p):
        return -imdiv(p)

class A:
    def __init__(self,):
        self.radon = Radon()
        self.grad = Grad()

    def __call__(self, u):
        return lv([self.radon(u), self.grad(u)])

    def adjoint(self, p):
        return self.radon.adjoint(p[0]) + self.grad.adjoint(p[1])


alpha = 1.0/(np.prod(phantom.shape[-2:]))
lamda = .5

if lamda > 0.:
    def prox_fconj(p, sigma):
        p_0 = alpha/(alpha + sigma) * p[0] - (sigma * alpha)/(alpha + sigma) * sinogram
        p_1 = lamda * p[1] / np.maximum(1, np.linalg.norm(p[1], axis=0)/lamda)[None,...] # projection on L-inf ball
        return lv([p_0, p_1])
else:
    def prox_fconj(p, sigma):
        p_0 = alpha/(alpha + sigma) * p[0] - (sigma * alpha)/(alpha + sigma) * sinogram
        p_1 = 0 * p[1]
        return lv([p_0, p_1])


def energy_fun(A, u):
    Au = A(u)
    return (alpha/2) * np.linalg.norm(Au[0] - sinogram)**2 + lamda * np.sum(np.abs(Au[1]))


x0 = ski.transform.iradon(sinogram, theta=theta)
#x0 = np.zeros_like(phantom)
y0 = A()(x0)

tau = 0.001
sigma = 0.01

n_iter = 200
#prox_fconj

In [3]:
from optimizer import lscg

def soft_shrinkage(x, lamda):
    return np.maximum(np.abs(x)-lamda, 0.) * np.sign(x)

class split_Bregman_TV:
    def __init__(self, A, rhs, u0, gamma=1.0, max_it=10, verbosity = 1):
        self.A = A
        self.gamma = gamma
        self.num_it = 0
        self.max_it = max_it
        self.verbosity = verbosity
        
        class cg_op:
            def __call__(self, u):
                return lv([gamma * A(u), 0.5 * imgrad(u)])

            def adjoint(self, p):
                return gamma * A.adjoint(p[0]) - 0.5 * imdiv(p[1])

        self.cg_op = cg_op()
        self.rhs = rhs

        self.u = u0
        self.b = imgrad(u0)
        self.d = soft_shrinkage(self.b + imgrad(self.u), self.gamma)

    def step(self):
        cg_rhs = lv([self.gamma * self.rhs, 0.5 * (self.d - self.b)])
        self.u = lscg(self.cg_op, cg_rhs, self.u).solve()
        self.d = soft_shrinkage(self.b + imgrad(self.u), self.gamma)
        self.b = self.b + imgrad(self.u) - self.d

    def terminate(self):
        if self.num_it > self.max_it:
            return True
        else:
            return False

    def solve(self,):
        while not self.terminate():
            self.step()
            if self.verbosity > 0:
                print('Iteration: ' + str(self.num_it))
        

In [4]:
sBTV = split_Bregman_TV(Radon(), sinogram, x0)
sBTV.solve()

C:\Users\roithtim\AppData\Local\miniconda3\lib\site-packages\skimage\transform\radon_transform.py:75: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:

opt = pdhg(x0=x0, y0=y0, K=A(), prox_fconj=prox_fconj, tau=tau, sigma=sigma, n_iter=n_iter, energy_fun = energy_fun)
res = opt.compute()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2)
im = ax[0].imshow(opt.x)
plt.colorbar(im, ax=ax[0])
ax[1].plot(opt.energy_hist)

In [ ]:
plt.imshow(x0)
plt.colorbar()

In [ ]:
def test_adjoint(A, x, y=None):
    Ax = A(x)
    if y is None:
        y = np.random.uniform(size=Ax.shape)
    res_1 = np.sum(Ax * y)
    res_2 = np.sum(x * A.adjoint(y))
    return res_1, res_2


In [ ]:
res_1, res_2 = test_adjoint(Grad(), phantom)

print(res_1)
print(res_2)
print((res_2/res_1))

In [ ]:
A = np.random.uniform(size = (20,20))

class B:
    def __call__(self, x):
        return A@x

    def adjoint(self,y):
        return A.T@y

In [ ]:
test_adjoint(B(), np.random.uniform(size=(20)))

In [ ]:
np.pi*np.sqrt(0.5)*3